In [1]:
import gym

In [29]:
import numpy as np
import operator
from collections import defaultdict

class SARSAAgent():
    def __init__(self, possibleActions,learningRate,discountFactor, epsilon, initVals=0.0):
        self.possibleActions = possibleActions
        self.learningRate = learningRate
        self.discountFactor = discountFactor
        self.epsilon = epsilon
        self.state = []
        self.G = initVals
        self.prevState = []
        self.prevAction = None
        self.prevReward = initVals
        self.currState = []
        self.currAction = None
        self.QValues = defaultdict(lambda : {})
        self.policy = defaultdict(float)
        self.numPossActions = len(self.possibleActions)
        

    def getPolicy(self):
        # Generate e-greedy policy
        if self.state in self.QValues.keys():
            for action in self.possibleActions:
                self.policy[action] = self.epsilon / self.numPossActions
            bestAction = max(self.QValues[self.state].items(), key=operator.itemgetter(1))[0] 
            self.policy[bestAction] += (1.0 - self.epsilon)  
        else:
            for action in self.possibleActions:    
                self.policy[action] = 1 / self.numPossActions
                
        #print("Policy to take action : ")
        #for k, v in self.policy.items():
         #   print("action ",k,"Choose prob ",v)


    def toStateRepresentation(self, state):
        return tuple((state[0],state[1]))
    

    def setExperience(self, state, action, reward):
        # Save data from t-1 - First step in episode
        if self.prevState == []:
            self.prevState = state      
            self.prevAction = action
            self.prevReward = reward
        
        self.currState = state
        self.currAction = action
        self.currReward = reward


    def setState(self, state):
        self.state = state    
        
    def reset(self):
        self.state = []
        self.prevState = []
        self.prevAction = None
        self.prevReward = 0.0
        self.currState = []
        self.currAction = None
        self.currReward = 0.0

    def act(self):
        # Take an action
        self.getPolicy()
        probs = list(self.policy.values())
        actions = list(self.policy.keys())
        action = actions[np.random.choice(np.arange(len(probs)), p=probs)]
        return action
    

    def setEpsilon(self, epsilon):
        self.epsilon = epsilon 

    def computeHyperparameters(self):
        return self.epsilon

    
    def learn(self):  
        
        print("-----Learning update-----")
        
        #Initialize Q[state][action] to zero
        if not self.currState in self.QValues.keys():
            for action in self.possibleActions:
                self.QValues[self.currState][action] = 0
                
        if not self.prevState in self.QValues.keys():
            for action in self.possibleActions:
                self.QValues[self.prevState][action] = 0
                
        # If action is none, step is terminal and Q[state][action] is zero.
        if self.currAction != None: 
            target = self.prevReward + (self.discountFactor * self.QValues[self.currState][self.currAction])    
        else: 
            target = self.prevReward 
        
        
        delta = target - self.QValues[self.prevState][self.prevAction]
        self.QValues[self.prevState][self.prevAction] += self.learningRate * delta
        
        # Save the current values for next time
        self.prevState = self.currState
        self.prevAction = self.currAction
        self.prevReward = self.currReward
        

        return self.QValues
        
       
                    
            
    
   

In [30]:

if __name__ == '__main__':

    env = gym.make('Blackjack-v1')
    space_size = env.action_space.n
    possibleActions = []
    for i in range(space_size):
        possibleActions.append(str(i))
        

    # Initialize a SARSA Agent
    agent = SARSAAgent(possibleActions,learningRate = 0.1, discountFactor = 0.999, epsilon = 0.9)

    # Run training SARSA Method
    for episode in range(30):
        print("\n******************************* EPISODE ",episode,"***************************")
        agent.reset()
        observation = env.reset()   # Returns current state
        nextObservation = None
        epsStart = True
        print("State after reset: ",observation)
       
        done = False
    
        while done==False:
            
            epsilon = agent.computeHyperparameters()
            agent.setEpsilon(epsilon)
            obsCopy = observation   # Copy current state
            agent.setState(agent.toStateRepresentation(obsCopy))
            action = agent.act()

            print("Action: ",action)
            
            nextObservation, reward, done, status = env.step(int(action))

            
            print("Next state: ",nextObservation)
            print("Next reward: ",reward)
            
            print("Episode finished: ",done)
            agent.setExperience(agent.toStateRepresentation(obsCopy), str(action), reward)
                        
            if not epsStart :
                agent.learn()
            else:
                epsStart = False
            
            observation = nextObservation
            

        agent.setExperience(agent.toStateRepresentation(nextObservation), None, None)
        agent.learn()

   


Possible actions:  ['0', '1']

******************************* EPISODE  0 ***************************
State after reset:  (15, 10, False)
Action:  1
Next state:  (23, 10, False)
Next reward:  -1.0
Episode finished:  True
-----Learning update-----

******************************* EPISODE  1 ***************************
State after reset:  (21, 7, True)
Action:  0
Next state:  (21, 7, True)
Next reward:  1.0
Episode finished:  True
-----Learning update-----

******************************* EPISODE  2 ***************************
State after reset:  (6, 7, False)
Action:  1
Next state:  (13, 7, False)
Next reward:  0.0
Episode finished:  False
Action:  1
Next state:  (23, 7, False)
Next reward:  -1.0
Episode finished:  True
-----Learning update-----
-----Learning update-----

******************************* EPISODE  3 ***************************
State after reset:  (21, 2, True)
Action:  0
Next state:  (21, 2, True)
Next reward:  1.0
Episode finished:  True
-----Learning update-----

******

In [31]:

if __name__ == '__main__':

    env = gym.make('Blackjack-v1')
    space_size = env.action_space.n
    possibleActions = []
    for i in range(space_size):
        possibleActions.append(str(i))
        

    # Initialize a SARSA Agent
    agent = SARSAAgent(possibleActions,learningRate = 0.1, discountFactor = 0.999, epsilon = 0.9)

    # Run training SARSA Method
    for episode in range(50000):
        agent.reset()
        observation = env.reset()   # Returns current state
        nextObservation = None
        epsStart = True

       
        done = False
    
        while done==False:
            
            epsilon = agent.computeHyperparameters()
            agent.setEpsilon(epsilon)
            obsCopy = observation   # Copy current state
            agent.setState(agent.toStateRepresentation(obsCopy))
            action = agent.act()

            
            nextObservation, reward, done, status = env.step(int(action))


            agent.setExperience(agent.toStateRepresentation(obsCopy), str(action), reward)
                        
            if not epsStart :
                agent.learn()
            else:
                epsStart = False
            
            observation = nextObservation
            

        agent.setExperience(agent.toStateRepresentation(nextObservation), None, None)

        agent.learn()

        QValues = agent.QValues


-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learnin

-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learnin

-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learnin

-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learnin

-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learnin

...

-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learnin

-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learnin

-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learnin

-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learnin

-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learning update-----
-----Learnin

In [18]:
    print("Q value function: ")
    m = 0
    for k, v in sorted(QValues.items()):
        m +=1
        print("item ",m,k,v)
        
    for k, v in sorted(QValues.items()):
        print("state: ",k," action: ",max(v.items(), key=operator.itemgetter(1))[0], " value: ",max(v.items(), key=operator.itemgetter(1))[1]) 
     


Q value function: 
item  1 (4, 1) {'0': -0.18953279, '1': -0.48175668990840625}
item  2 (4, 2) {'0': 0.10339834914710003, '1': -0.25135324371911577}
item  3 (4, 3) {'0': -0.0339394272320359, '1': -0.15694606855094184}
item  4 (4, 4) {'0': -0.09786209567788613, '1': -0.16044734034883237}
item  5 (4, 5) {'0': -0.011037235699000017, '1': -0.23588163035998738}
item  6 (4, 6) {'0': -0.174920131, '1': -0.1253847270026015}
item  7 (4, 7) {'0': 0.02256628411000003, '1': -0.2598445771369511}
item  8 (4, 8) {'0': -0.24543207545039006, '1': -0.32006096750361845}
item  9 (4, 9) {'0': -0.039240999999999984, '1': -0.3036819188693759}
item  10 (4, 10) {'0': -0.588011700403901, '1': -0.514405781093053}
item  11 (5, 1) {'0': -0.3978073039899978, '1': -0.6082549638117379}
item  12 (5, 2) {'0': -0.40796909381171964, '1': -0.323271660923643}
item  13 (5, 3) {'0': -0.11596026615345358, '1': -0.2886455177684539}
item  14 (5, 4) {'0': 0.0884662647479641, '1': -0.25569119901863485}
item  15 (5, 5) {'0': -0.12

state:  (15, 9)  action:  1  value:  -0.7794540805802395
state:  (15, 10)  action:  0  value:  -0.6054117009818614
state:  (16, 1)  action:  1  value:  -0.6928935182691423
state:  (16, 2)  action:  0  value:  0.007005553100019601
state:  (16, 3)  action:  0  value:  -0.02490840080887638
state:  (16, 4)  action:  0  value:  -0.4379292643393484
state:  (16, 5)  action:  0  value:  -0.1704610549845077
state:  (16, 6)  action:  0  value:  -0.522600438508518
state:  (16, 7)  action:  0  value:  -0.5729279206345085
state:  (16, 8)  action:  0  value:  -0.41345512797433837
state:  (16, 9)  action:  0  value:  -0.5603370674494879
state:  (16, 10)  action:  0  value:  -0.41319251719730754
state:  (17, 1)  action:  0  value:  -0.7671167333769146
state:  (17, 2)  action:  0  value:  -0.44453990214537564
state:  (17, 3)  action:  0  value:  -0.15358543213754033
state:  (17, 4)  action:  1  value:  -0.3135934794315948
state:  (17, 5)  action:  0  value:  0.029233478363785884
state:  (17, 6)  action